# Survival Game 
<font size=4 color='gray'>Alexander Bruun, Anton Paris </font>
# <font color='gray'>A Predator Prey Simulation </font>
Based on Leonel Aguilar's 2D Cellular Automata Code
***




## Overview    <font size=3 color='gray'> -  Table of content for faster Navigation </font>
***

-  [Import Libraries](#import)   
-  [User Imput](#user) <br>
&ensp; [Meaning of the index in Lists](#meaning_list)
-  [Class Cell](#cell) <br>
&ensp; [Properties of the Class Cell](#properties)
-  [section title](#section-title)  
-  [section title](#section-title)  
-  [section title](#section-title)  

<a id=''></a>

<a id='import'></a>

### Import Libraries


In [1]:
import numpy as np #matrix and vector library
import matplotlib.pyplot as plt #plotting library
from random import shuffle
import random 
import matplotlib.animation
from copy import deepcopy
import pandas as pd

In [2]:
%matplotlib inline 

<a id='user'></a>

### User Imput 


##### Use `CTRL + F`  to find wanted variables  
***
##### Variables that have an great impact on the simulation are noted here 
Variables: <br>
`r_dim ` amount of rows<br>

`c_dim`  amount of columns<br>

`offspring_chance_table` chance for offspring<br>

`rules` List of priority for each animal (Where does the animal want to go) <br>

***

 <a id="meaning_list"></a>
#### Meaning of the index in Lists  
Sets parameters for Cells with the state equal to the index! <br> 
<b>list_name[0]=Forest, [1]=Rabbits, [2]=Wolves, [3]=Deers ,[4]=Bear </b>
   


In [3]:
global r_dim, c_dim, cells
#
r_dim= 100
c_dim= 100
cells=[]



In [4]:
#Set wanted parameters

#0 means 0% chance, 100 means 100%  probability for offspring.
#Predators do have 100% because other rules will be applied to them 
offspring_chance_table=[0,30,100,20,100] 

# Contains the information how high the nutrition_level of an animal has to be to reproduce
values_offspring=[0,0,20,0,40]

#values represent nutrition level boost they get for EATING certain animal
#meaning if a bear [4] eats a wolf [2] the bear gets the value in [2]
eat_values=[0,7,10,15,30]

#Values represent the ageing of an animal, value is added to death_probability each turn 
death_values=[0,5,10,2,1]

#Defines the behavior while looking for possible movement
#Bear [4] prefers [rabbit [1], deer[3], Wolf[2], Forest [0]]
rules= [[0],[0],[3,1,0],[0],[1,3,2,0]]





<a id='cell'></a>

### Create Class Cell

<a id="properties"></a>

#### Properties of the class Cell. 

`type_cell` :   Forest, [1]=Rabbits, [2]=Wolves, [3]=Deers ,[4]=Bear<br>
`lifespan` :  how many turns the animial is alife <br> 
`nutition_level`:  Energy level, increases when animal eats something decreases when it moves <br>
`reproduction_cont`: Counts Offspring<br>
`death_probability`: Probability that the animal dies in a round, increases steady each turn <br>
`turn` True if it already had it's turn in the timestep<br>
`offspring_chance` Chance for mating<br>



In [5]:
class Cell():
    def __init__(self, type_cell, nutrition_level):
        self.type_cell = type_cell
        self.lifespan = 0
        self.nutrition_level= nutrition_level
        self.reproduction_count=0
        self.death_probability=1
        self.turn=False
        self.offspring_chance=self.offspring_chance_table[type_cell]

    #PRE: 
    #POST:
    #USED FOR:    
    def get_type (self):
        return self.type_cell
    
    #PRE: Value of the wanted increase of lifespan
    #POST: Increases the lifespan of the animal by value
    #USED FOR: Eat Mechanism 
    def increase_l (self, value):
        self.lifespan+= value   
       # print(self.lifespan, "self.lifespan")
        
#TURN FUNCTIONS 

    #PRE: -
    #POST: turn= true
    #USED FOR: Limit the number of turns in a timestep to 1
    def set_turn(self):
        self.turn=True
       # print(self.turn,"after set turn ")
        
    #PRE: -
    #POST: turn= false
    #USED FOR: Start a the round to reset the turn boolean to false: Cell is allowed to apply rules
    def round_start(self):
       # print(self.turn,"before round start")
        self.turn=False
        
     
 #CHANGE CELLS FUNCTIONS
    
           
        
    #PRE: value of the nutrition_level that will be gained (can also be 0 if forest)
    #POST: increased nutrition_level
    #USED FOR: perform nutrition gain by eating an animal/forest
    def eat (self, value):
        self.nutrition_level+= value
        

    #PRE: Coordinates of a second Cell where the prime Cell(currently evaluated ) 
    #POST: dublicates attributes to the Cell with the Coordinates passed (coords2)
    #USED FOR: Part of the movement function, simulate a jump to a new cell (with coords 2)
    def copy_attributes(self, coords_2):
       # print("copying attributes,", cells[coords_2[0]][coords_2[1]].type_cell,self.type_cell)
        cells[coords_2[0]][coords_2[1]].type_cell=self.type_cell
        cells[coords_2[0]][coords_2[1]].lifespan = self.lifespan
        cells[coords_2[0]][coords_2[1]].nutrition_level = self.nutrition_level
        cells[coords_2[0]][coords_2[1]].turn= self.turn 
        cells[coords_2[0]][coords_2[1]].reproduction_count=self.reproduction_count
        cells[coords_2[0]][coords_2[1]].death_probability=self.death_probability
        
    #PRE:-
    #POST: returns lifepsan, nutrition_level, reproduction count 
    #USED FOR:getting information of the cell   
    def get_attributes(self):
        return self.lifespan, self.nutrition_level, self.reproduction_count
    
    
    #PRE:-
    #POST: increase reproduction counter 
    #USED FOR: offspirng mechanism 
    def offspring(self):
        self.reproduction_count+=1
        
    #PRE:-
    #POST: Sets a cell to type Forest and resetts attributes of the cell
    #USED FOR: simulating death      
    def death (self):
        self.lifespan=0
        #print(cells[coords[0]][coords[1]].type_cell," before")
        self.type_cell=0 #forest
        self.nutrition_level=0
        self.reproduction_count=0
        self.death_probability=1
        #print(cells[coords[0]][coords[1]].type_cell," after")
     
    
    #PRE:-
    #POST: returns bool of random death (age based)
    #USED FOR: random death because of age 
    def death_age(self):
        return (random.randint (1,100) < self.death_probability)
                 
     
    #PRE: death_values is a list with 5 values, index of Values must represent the different cell types 
    #POST: returns the ageing value for the called cell
    #USED FOR: Information of the ageing process of each individual animal 
    def get_value_death_age(self,death_values):
        return death_values[self.type_cell]
    
        
    #PRE: friends is an integer representing how many cells with the same cell type are in the neighborhood
    #POST: Returns true or false, true= has the right to reproduce, false has not the right to reproduce 
    #USED FOR: Figuring out, if the Cell has the right to reproduce 
    def offspring(self,friends,values_offspring):
        #Rabbit, 
        if self.type_cell==1:
             return (friends and self.nutrition_level>=values_offspring[self.type_cell] and random.randint(1,100)<self.offspring_chance)
        #Deer
        elif self.type_cell==3:
             return (friends and self.nutrition_level>=values_offspring[self.type_cell] and random.randint(1,100)<self.offspring_chance)
        #Wolf, Bear and 
        else:
            return (friends and self.nutrition_level>=values_offspring[self.type_cell])

    #PRE: 2D List filled with Object of the class cell(cells), Coordinations of mohercell (after movement)
    #POST: Cell becomes a dublicate of the mothercell with reseted stts
    #USED FOR:    
    def birth(self,coords_2,cells):
        self.lifespan=1
        self.nutrition_level=5
        self.reproduction_count=0
        self.death_probability=1
        cells[coords_2[0]][coords_2[1]].reproduction_count+=1
        cells[coords_2[0]][coords_2[1]].nutrition_level-=10
        print(self.type_cell, "type_cell birth", self.nutrition_level, "nutritionlevel birth")

        
    #PRE:
    #POST:
    #USED FOR: 
    def set_movement(self, cells, coords_1, coords_2, increase_value_lifespan, increase_value_eat, friends):
        #print(coords_1,coords_2)
        
       
        #print("coords_2",cells[coords_2[0]][coords_2[1]].type_cell," coords_1",self.type_cell )
        
        #cells[coords_2[0]][coords_2[1]]=cells[coords_1[0]][coords_1[1]]
        #print("coords_2",cells[coords_2[0]][coords_2[1]].type_cell," coords_1",cells[coords_1[0]][coords_1[1]].type_cell )
        #print(self.turn, "does it function")
        if self.turn==False:
            
            self.set_turn()
            if self.type_cell==0:
                return 
            if self.type_cell==1 or self.type_cell==3:
                self.nutrition_level+=8
            elif self.type_cell!=0 and self.death_age():
                self.death()
              #  print("death by age")
                return 
            
                self.increase_l(increase_value_lifespan)
                self.eat(increase_value_eat)
                self.death_probability+=self.get_value_death_age()
        
            if coords_1!=coords_2:
                if self.type_cell!=0 and self.type_cell!=3:
                    self.nutrition_level-=2
                self.copy_attributes(coords_2)
                if self.offspring(friends)and self.nutrition_level>0 :
                    self.birth(coords_2,cells)
                else:
                    self.death()
               # print("death by movement")
            #print("coords_2",cells[coords_2[0]][coords_2[1]].type_cell," coords_1",cells[coords_1[0]][coords_1[1]].type_cell )
            
            #print("from: ", coords_1, "to: ", coords_2)
        return
    
